In [ ]:
import torchvision.transforms as transforms
import torch
import torch.utils.data as data
import numpy as np

from trainDataset import TrainDataset
from testDataset import TestDataset

toFloat = transforms.Lambda(lambda x: x / np.iinfo(np.int16).max)

trainDataset = TrainDataset("../kaggle-train", transform=toFloat)
print(len(trainDataset))

testDataset = TestDataset("../kaggle-test", transform=toFloat)
print(len(testDataset))

In [ ]:
from sklearn import svm

familyClassifier = svm.SVC()
sourceClassifier = svm.SVC()

trainLoader = data.DataLoader(trainDataset, batch_size=3200, shuffle=True)
for samples, instrument_family_target, instrument_source_target in trainLoader:
    familyClassifier.fit(np.array(samples.data)[:, :16000], np.array(instrument_family_target.data))
    sourceClassifier.fit(np.array(samples.data)[:, :16000], np.array(instrument_source_target.data))
    break

In [ ]:
batch_size = 32
testloader = data.DataLoader(testDataset, batch_size=batch_size, shuffle=False)

familyPredictions = np.zeros(len(testDataset), dtype=np.int)
sourcePredictions = np.zeros(len(testDataset), dtype=np.int)

for index, samples in enumerate(testloader):
    familyPredictions[index*batch_size:(index+1)*batch_size] = familyClassifier.predict(np.array(samples.data)[:, :16000])
    sourcePredictions[index*batch_size:(index+1)*batch_size] = sourceClassifier.predict(np.array(samples.data)[:, :16000])


In [ ]:
import csv

familyPredictionStrings = trainDataset.transformInstrumentTargetsToString(familyPredictions.astype(int))
sourcePredictionStrings = trainDataset.transformSourcesTargetsToString(sourcePredictions.astype(int))

with open('SVM-submission.csv', 'w', newline='') as writeFile:
    fieldnames = ['Id', 'Expected']
    writer = csv.DictWriter(writeFile, fieldnames=fieldnames, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writeheader()
    for index in range(len(testDataset)):
        writer.writerow({'Id': index, 
                         'Expected': familyPredictionStrings[index] + '_' + sourcePredictionStrings[index]})


In [ ]:
familyPredictionStrings

In [ ]:
sourcePredictionStrings